In [1]:
spark
import numpy as np
import pandas as pd
import re
import pyspark
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import trim,ltrim,rtrim
from pyspark.sql.functions import *
from pyspark.sql.functions import mean
from ydata_profiling import ProfileReport
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import lit
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql.functions import mean, when, col
from pyspark.ml.classification import DecisionTreeClassificationModel
from pycaret.classification import *
from pycaret.classification import ClassificationExperiment
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

Intitializing PySpark ...

23/05/30 19:06:25 WARN Utils: Your hostname, DESKTOP-VJTDOFP resolves to a loopback address: 127.0.1.1; using 172.27.220.32 instead (on interface eth0)
23/05/30 19:06:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/05/30 19:06:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/30 19:06:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/30 19:06:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Spark Web UI available at http://172.27.220.32:4042
SparkContext available as 'sc' (version = 3.3.1, master = local[*], app id = local-1685469987549)
SparkSession available as 'spark'


In [2]:
path = "./model_data/part-00000-8f3d2c8d-e21b-4ae6-a337-4cbd28bb3ac8-c000.csv"
df = spark.read.option('header' , 'True').csv(path)
df = df.withColumn("Prize",df.Prize.cast('integer'))
df = df.withColumn("nSets",df.nSets.cast('integer'))
df = df.withColumn("Year",df.Year.cast('integer'))
df = df.withColumn("rank_dif",df.rank_dif.cast('double'))
df = df.withColumn("loser_home",df.loser_home.cast('int'))
df = df.withColumn("winner_home",df.winner_home.cast('int'))
df = df.withColumn("GameRank_winner",df.GameRank_winner.cast('double'))
df = df.withColumn("GameRank_loser",df.GameRank_loser.cast('double'))
df = df.withColumn("Height_dif",df.Height_dif.cast('double'))



In [3]:
model_df_auto = df

In [7]:
dataframes = model_df_auto.randomSplit([0.7, 0.3], seed=26)

In [8]:
indexerground = StringIndexer(inputCol="Ground", outputCol="GroundIndex")
encoderground = OneHotEncoder(inputCol="GroundIndex", outputCol="GroundVec")
indexer_gameround = StringIndexer(inputCol="GameRound", outputCol="GameRoundIndex")
encoder_gameround = OneHotEncoder(inputCol="GameRoundIndex", outputCol="GameRoundVec")

In [10]:
feature_cols = ["Prize", "GameRank_winner", "GameRank_loser", "GroundVec", "Year", "GameRoundVec"]
label_col = "nSets"
assembler = VectorAssembler(
    inputCols=["Prize", "GameRank_winner", "GameRank_loser", "GroundVec", "GameRoundVec"],
    outputCol="features", handleInvalid="keep")
pipeline = Pipeline(stages=[indexerground, indexer_gameround, encoderground, encoder_gameround, assembler])
pipeline_model = pipeline.fit(model_df_auto)
train_data = pipeline_model.transform(dataframes[0]).select("features", label_col)
test_data = pipeline_model.transform(dataframes[1]).select("features", label_col)


In [11]:
dt = DecisionTreeClassifier(featuresCol="features", labelCol="nSets")
dt_model = dt.fit(train_data)

In [17]:
dt_model = dt.fit(train_data)
predictions = dt_model.transform(test_data)
predictions = predictions.select("prediction", "nSets")
arvorefixe = dt_model
print(arvorefixe.toDebugString)


DecisionTreeClassificationModel: uid=DecisionTreeClassifier_f2775db3fe53, depth=5, numNodes=25, numClasses=4, numFeatures=17
  If (feature 2 <= 944.5142857142857)
   If (feature 2 <= 224.91071428571428)
    Predict: 2.0
   Else (feature 2 > 224.91071428571428)
    If (feature 1 <= 740.5)
     Predict: 2.0
    Else (feature 1 > 740.5)
     If (feature 14 in {0.0})
      Predict: 2.0
     Else (feature 14 not in {0.0})
      If (feature 2 <= 318.5)
       Predict: 2.0
      Else (feature 2 > 318.5)
       Predict: 3.0
  Else (feature 2 > 944.5142857142857)
   If (feature 1 <= 655.5)
    If (feature 10 in {0.0})
     Predict: 2.0
    Else (feature 10 not in {0.0})
     If (feature 2 <= 1457.2692307692307)
      If (feature 1 <= 86.5)
       Predict: 3.0
      Else (feature 1 > 86.5)
       Predict: 2.0
     Else (feature 2 > 1457.2692307692307)
      Predict: 2.0
   Else (feature 1 > 655.5)
    If (feature 6 in {0.0})
     Predict: 2.0
    Else (feature 6 not in {0.0})
     If (feature 0 

In [25]:
um = predictions.where((predictions.nSets == 2)&(predictions.prediction==2)).count()
um

15305

In [26]:
dois = predictions.where((predictions.nSets == 3)&(predictions.prediction==3)).count()
dois

2

In [27]:
tres = predictions.where((predictions.nSets == 2)&(predictions.prediction==3)).count()
tres

10

In [28]:
quatro = predictions.where((predictions.nSets == 3)&(predictions.prediction==2)).count()
quatro

7506

In [29]:
(um+dois)/(um+dois+tres+quatro)

0.6706830828550147